# Programação Matemática (GCC118)
## Universidade Federal de Lavras (UFLA)
### Instituto de Ciências Exatas e Tecnológicas

#### Grupo
- Ranulfo Mascari Neto
- Heitor Rodrigues Sabino

# Atividade Prática 2

## Problema 1: Empresa de relógios

### Descrição

João, Ana e Lídia são os únicos sócios e trabalhadores em uma empresa que produz relógios de primeira qualidade. João e Ana podem dedicar no máximo **40 horas por semana** (cada um) à empresa, enquanto Lídia tem disponibilidade de, no máximo, **20 horas semanais**.

A empresa fabrica dois tipos de relógios: **relógio de pedestal** (modelo antigo) e **relógio de parede**.

#### Descrição das Atividades

- **João** (engenheiro): Monta as peças mecânicas internas do relógio.
- **Ana** (carpinteira): Produz as caixas de madeira esculpidas à mão.
- **Lídia**: Responsável pelas encomendas e pela remessa dos relógios.

#### Tempo Necessário para Produção

| Tarefa                     | Relógio de Pedestal | Relógio de Parede |
|----------------------------|---------------------|-------------------|
| Montagem do mecanismo      | 6h                 | 4h               |
| Caixa de madeira esculpida | 8h                 | 4h               |
| Remessa                    | 3h                 | 3h               |

#### Lucro Gerado por Relógio

- Cada **relógio de pedestal** construído e despachado gera lucro de **R\$ 300,00**.
- Cada **relógio de parede** gera lucro de **R\$ 200,00**.

#### Objetivo

Os três sócios querem determinar **quantos relógios de cada tipo devem ser produzidos semanalmente** para **maximizar o lucro total**.

### 1. Implemente um modelo de programação linear, usando o *gurobipy*, e encontre a solução ótima do problema.

#### Modelagem Matemática
A seguir, apresentaremos a modelagem matemática deste problema, especificando os principais elementos da modelagem de um problema de programação matemática: $(i)$ parâmetros (dados); $(ii)$ variáveis de decisão; $(iii)$ modelagem, composta por uma função objetivo, restrições do problema e restrições de domínio das variáveis de decisão.

##### $(i)$ Parâmetros

- **Lucro por Relógio**:
  - $ l_r \in \mathbb{R}_+ $: lucro (em R\$) por unidade do relógio do tipo $ r $, onde $ r \in R $.
    - $ l_{pedestal} = 300 $
    - $ l_{parede} = 200 $

- **Tempo Necessário para Produção**:
  - $ t_{w,r} \in \mathbb{R}_+ $: tempo necessário (em horas) para o trabalhador $ w $ produzir uma unidade do relógio do tipo $ r $, onde $ w \in W $ e $ r \in R $.
    - **João** ($ w = joao $):
      - $ t_{joao,pedestal} = 6 $
      - $ t_{joao,parede} = 4 $
    - **Ana** ($ w = ana $):
      - $ t_{ana,pedestal} = 8 $
      - $ t_{ana,parede} = 4 $
    - **Lídia** ($ w = lidia $):
      - $ t_{lidia,pedestal} = 3 $
      - $ t_{lidia,parede} = 3 $

- **Disponibilidade Máxima de Tempo**:
  - $ d_w \in \mathbb{R}_+ $: disponibilidade máxima de horas semanais do trabalhador $ w $, onde $ w \in W $.
    - $ d_{joao} = 40 $
    - $ d_{ana} = 40 $
    - $ d_{lidia} = 20 $

- **Conjuntos**:
  - **Tipos de Relógio**:
    - $ R = \{pedestal, parede\} $: conjunto dos tipos de relógios fabricados.
  - **Trabalhadores**:
    - $ W = \{joao, ana, lidia\} $: conjunto de trabalhadores responsáveis pela produção.

##### $(ii)$ Variáveis de Decisão

- $ x_r \in \mathbb{R}_+ $: quantidade de relógios do tipo $ r $ a serem produzidos, onde $ r \in R $.
  - $ x_{pedestal} $: quantidade de relógios de pedestal produzidos.
  - $ x_{parede} $: quantidade de relógios de parede produzidos.
  - *Observação: No contexto real, as quantidades de relógios produzidos ($x_r$) deveriam ser inteiras, ou seja, $x_r \in \mathbb{Z}_+$.*

##### $(iii)$ Modelagem

- **Função Objetivo**:
  - **Maximizar o Lucro Total**:
  $$
  \max \sum_{r \in R} l_r \cdot x_r
  $$
    - Maximiza o lucro total gerado pela produção dos relógios.

- **Restrições**:
  1. **Tempo Disponível para Cada Trabalhador**:
    $$
    \sum_{r \in R} t_{w,r} \cdot x_r \leq d_w, \quad \forall w \in W
    $$
    - Garante que o tempo gasto na produção por cada trabalhador não exceda sua disponibilidade semanal.

  2. **Não Negatividade**:
    $$
    x_r \geq 0, \quad \forall r \in R
    $$
    - As quantidades de relógios a serem produzidas devem ser não negativas.

---

#### Modelo Matemático do Problema

##### **Função Objetivo**
**Maximizar o Lucro Total**:
$$
\max 300 \cdot x_{pedestal} + 200 \cdot x_{parede}
$$

Essa função maximiza o lucro total gerado pela produção de relógios de pedestal e parede, considerando os lucros por unidade.

##### **Restrições**
1. **Tempo Disponível para João**:
$$
6 \cdot x_{pedestal} + 4 \cdot x_{parede} \leq 40
$$
  - O tempo total gasto por João na produção de ambos os tipos de relógios não pode exceder sua disponibilidade semanal de 40 horas.

2. **Tempo Disponível para Ana**:
$$
8 \cdot x_{pedestal} + 4 \cdot x_{parede} \leq 40
$$
  - Ana também possui 40 horas disponíveis semanalmente para produção.

3. **Tempo Disponível para Lídia**:
$$
3 \cdot x_{pedestal} + 3 \cdot x_{parede} \leq 20
$$
  - Lídia tem apenas 20 horas semanais disponíveis, limitando sua capacidade produtiva.

4. **Não Negatividade**:
$$
x_{pedestal} \geq 0, \quad x_{parede} \geq 0
$$
  - As quantidades de relógios de pedestal e parede a serem produzidas devem ser números não negativos.

#### Resolução do Problema

#### Instalação da biblioteca Gurobi

In [1]:
!pip install gurobipy

In [2]:
from gurobipy import Model, GRB

# Conjuntos
R = ["Pedestal", "Parede"] # Tipos de relógios
W = ["João", "Ana", "Lídia"] # Trabalhadores

# Parâmetros
l = {
    "Pedestal": 300,
    "Parede": 200
}

t = {
    "João": {
        "Pedestal": 6,
        "Parede": 4
    },
    "Ana": {
        "Pedestal": 8,
        "Parede": 4
    },
    "Lídia": {
        "Pedestal": 3,
        "Parede": 3
    }
}

d = {
    "João": 40,
    "Ana": 40,
    "Lídia": 20
}

# Criação do modelo
model = Model("Maximização do Lucro - Produção de Relógios")

# Variáveis de decisão
x = {r: model.addVar(vtype=GRB.CONTINUOUS, name=f"x_{r}") for r in R}

# Função objetivo: Maximizar o lucro total
model.setObjective(
    sum(l[r] * x[r] for r in R),
    GRB.MAXIMIZE
)

# Restrições de tempo para cada trabalhador
for w in W:
    model.addConstr(
        sum(t[w][r] * x[r] for r in R) <= d[w],
        name=f"d_{w}"
    )

# Resolução do modelo
model.optimize()

# Resultados
if model.status == GRB.OPTIMAL:
    print("\nSolução Ótima:")
    print("\nVariáveis de Decisão:")
    for r in R:
        print(f"\t{r}: {x[r].x}")
    print(f"\tLucro Total: R$ {model.objVal:.2f}")
else:
    print("Não foi encontrada solução ótima.")


Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 3 rows, 2 columns and 6 nonzeros
Model fingerprint: 0x6103ff60
Coefficient statistics:
  Matrix range     [3e+00, 8e+00]
  Objective range  [2e+02, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 4e+01]
Presolve time: 0.01s
Presolved: 3 rows, 2 columns, 6 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.0000000e+32   4.250000e+30   5.000000e+02      0s
       2    1.6666667e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.666666667e+03

Solução Ótima:

Variáveis de Decisão:
	Pedestal: 3.333333333333333
	Parede: 3.3333333333333335
	Lucr

### 2. Faça a análise de sensibilidade do modelo implementado, utilizando o *gurobipy*. Em sua resposta, **além de apresentar os valores, apresente uma explicação sobre o que os resultados indicam.**

#### Análise de Sensibilidade

##### Implementação da Análise de Sensibilidade com a Biblioteca *GurobiPy*

In [3]:
# Análise de Sensibilidade

# O modelo atingiu uma solução ótima
if model.status == GRB.OPTIMAL:

  # Valores das variáveis duais (Preços Sombra)
  print("\nSolução Ótima Dual:")
  for c in model.getConstrs():
      print(f"\t{c.constrName} - Valor Dual (Pi): {c.Pi}")

  # Intervalos de viabilidade das tendências para o vetor de recursos
  print("\nIntervalos de Recursos (Vetor de Recursos):")
  for c in model.getConstrs():
      print(f"\t{c.constrName} [SARHSLower: {c.SARHSLow}, SARHSUpper: {c.SARHSUp}]")

  # Intervalos de viabilidade das tendências para o vetor de custos
  print("\nIntervalos de Custos (Vetor de Custos):")
  for v in model.getVars():
      print(f"\t{v.varName} [SAObjLow: {v.SAObjLow}, SAObjUp: {v.SAObjUp}]")


Solução Ótima Dual:
	d_João - Valor Dual (Pi): 0.0
	d_Ana - Valor Dual (Pi): 25.0
	d_Lídia - Valor Dual (Pi): 33.333333333333336

Intervalos de Recursos (Vetor de Recursos):
	d_João [SARHSLower: 33.333333333333336, SARHSUpper: inf]
	d_Ana [SARHSLower: 26.666666666666668, SARHSUpper: 53.33333333333333]
	d_Lídia [SARHSLower: 15.0, SARHSUpper: 30.0]

Intervalos de Custos (Vetor de Custos):
	x_Pedestal [SAObjLow: 200.0, SAObjUp: 400.0]
	x_Parede [SAObjLow: 150.0, SAObjUp: 300.0]


##### **Interpretação da Solução Ótima Dual**

A **solução ótima dual** fornece informações sobre os valores das **variáveis duais** ($\lambda$) associados às restrições do modelo. Esses valores indicam como o lucro total seria afetado por uma pequena alteração na disponibilidade de tempo ($d_w$) de cada trabalhador.

###### Resultados
1. **João ($d_{\text{João}}$)**:
   - **Valor Dual ($\lambda_1 = 0.0$)**
   - **Interpretação**:
     - O tempo disponível de João ($d_{\text{João}}$) **não é uma restrição ativa** na solução ótima.
     - Isso significa que João ainda possui tempo ocioso, e aumentar ou reduzir sua disponibilidade **não impactará o lucro total**.

2. **Ana ($d_{\text{Ana}}$)**:
   - **Valor Dual ($\lambda_2 = 25.0$)**
   - **Interpretação**:
     - O tempo disponível de Ana ($d_{\text{Ana}}$) **é uma restrição ativa**, ou seja, todo o tempo de Ana está sendo utilizado.
     - Cada hora adicional disponível para Ana aumentaria o lucro total em **R\$ 25,00**:
       $$
       \frac{\partial F(d)}{\partial d_{\text{Ana}}} = \lambda_2 = 25
       $$

3. **Lídia ($d_{\text{Lídia}}$)**:
   - **Valor Dual ($\lambda_3 = 33.33$)**
   - **Interpretação**:
     - O tempo disponível de Lídia ($d_{\text{Lídia}}$) **é uma restrição ativa**, com todo o tempo sendo utilizado.
     - Cada hora adicional disponível para Lídia aumentaria o lucro total em **R\$ 33,33**:
       $$
       \frac{\partial F(d)}{\partial d_{\text{Lídia}}} = \lambda_3 = 33.33
       $$

##### **Análise de Sensibilidade à Perturbação no Vetor de Recursos**

Os intervalos de variação de recursos indicam os valores mínimos (`SARHSLower`) e máximos (`SARHSUpper`) para o lado direito das restrições (RHS), vetor de recursos, dentro dos quais a solução básica do modelo permanecerá factível, garantindo que as *tendências* não se alterem. Dentro desses limites, a estrutura da solução ótima, como as variáveis básicas e não-básicas, permanecerá inalterada.

**Disponibilidade Máxima de Tempo**: $ d_w \in \mathbb{R}_+ $: disponibilidade máxima de horas semanais do trabalhador $ w $, onde $ w \in W $.
- **João**: $d_{\text{João}} \in [33.33, \infty]$
  - A disponibilidade de João pode ser reduzida até 33.33 horas sem impactar a solução ótima. Não há limite superior para aumento.

- **Ana**: $d_{\text{Ana}} \in [26.67, 53.33]$
  - A disponibilidade de Ana deve permanecer entre 26.67 e 53.33 horas para que a solução ótima seja preservada.

- **Lídia**: $d_{\text{Lídia}} \in [15.0, 30.0]$
  - Lídia possui os limites mais restritos, indicando que pequenas alterações em sua disponibilidade podem alterar a solução ótima.

##### Análise de Sensibilidade à Perturbação no Vetor de Custos

### **Análise de Sensibilidade à Perturbação no Vetor de Custos**

Os intervalos de sensibilidade ao custo indicam os valores mínimos (`SAObjLow`) e máximos (`SAObjUp`) para os coeficientes do vetor objetivo (lucros unitários das variáveis de decisão), dentro dos quais a solução básica do modelo permanece ótima. Dentro desses limites, a estrutura da solução ótima, incluindo as variáveis básicas e não-básicas, não será alterada.

**Lucro Unitário**: $l_r \in \mathbb{R}_+$: lucro (em R\$) por unidade do relógio do tipo $r$, onde $r \in R$.

- **Relógio de Pedestal**: $l_{\text{Pedestal}} \in [200.0, 400.0]$
  - O lucro unitário do relógio de pedestal pode variar entre 200,00 e 400,00 reais sem impactar a solução ótima

- **Relógio de Parede**: $l_{\text{Parede}} \in [150.0, 300.0]$
  - O lucro unitário do relógio de parede pode variar entre 150,00 e 300,00 reais sem impactar a solução ótima.


***Observação***: *Os valores apresentados na análise foram arredondados para duas casas decimais, visando facilitar a interpretação e a clareza dos resultados.*

### 3. Para aumentar o lucro total, os três sócios concordaram que um deles aumentará ligeiramente o número máximo de horas disponíveis por semana. A escolha de qual deles se baseará naquele que aumentaria mais o lucro total. Use os resultados da análise de sensibilidade para fazer essa escolha.

Para determinar qual sócio deve aumentar ligeiramente o número máximo de horas disponíveis por semana e maximizar o impacto no lucro total, utilizamos os **valores duais ($\lambda$)** fornecidos na solução ótima dual. Esses valores indicam o aumento no lucro total para cada unidade adicional (hora) do recurso disponibilizado por um sócio, desde que o aumento permaneça dentro dos intervalos de sensibilidade (`SARHSLower` e `SARHSUpper`).

A sócia **Lídia** é a escolha ideal para aumentar o número de horas disponíveis, pois ela apresenta o maior valor dual ($\lambda_3 = 33.33$), indicando que cada hora adicional trabalhada por ela aumentará o lucro total em R\$ 33,33.

Além disso, a análise de sensibilidade ($15.0 \leq d_{\text{Lídia}} \leq 30.0$) revela que há margem para aumentar sua disponibilidade dentro desse intervalo sem alterar a estrutura ótima da solução, tornando essa decisão a mais eficiente para maximizar o lucro da empresa.


### 4. Existe alguma variável dual igual a zero? O que significa ter uma variável dual igual a zero?

Sim, a variável dual associada ao tempo de João ($d_{\text{João}}$) é igual a zero, ou seja, $\lambda_1 = 0$.

Uma variável dual igual a zero ($\lambda_i = 0$) indica que a restrição correspondente **não está ativa** na solução ótima. Em termos práticos:
  - O recurso associado à restrição (no caso, o tempo disponível de João, $d_{\text{João}}$) **não está completamente utilizado**.
  - Isso significa que João ainda tem **tempo ocioso** na solução ótima, e aumentar ou reduzir sua disponibilidade **não impactará o lucro total**.
